## Neural Net

In [1]:
import warnings
import time
from tqdm import tqdm
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning

from _util.custom_plotting import corr_heatmap, histogram_boxplot, horizontal_bar, heatmap_boxplot, simple_bar
from _util.make_confusion_matrix import make_cm
from _util.model_comparisons import *
from _util.custom_mem_opt import custom_mem_opt

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import PrettyPrinter

import gc

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn

pd.options.mode.chained_assignment = None
pp = PrettyPrinter(width=41, compact=True)
root = './_pkls/'

### Set some options

In [2]:
warnings.simplefilter(action='ignore', category=NumbaDeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
np.seterr(divide = 'ignore') 

# Ensure that the current MacOS version is at least 12.3+, and 
# the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_available(), ", ", torch.backends.mps.is_built())

pd.options.mode.chained_assignment = None  # default='warn'
%matplotlib inline

pp = PrettyPrinter(width=100)

True ,  True
